In [6]:
import pandas as pd
import plotly.express as px  # For Plotly Express, which is easy-to-use for quick visualizations
import plotly.graph_objects as go 

In [7]:
import seaborn as sns
import matplotlib.pyplot as plt

HOUSING RENTAL INCOME FORMATTING
---

In [24]:
housing = pd.read_excel("../data/raw/week_3_project_data.xlsx", sheet_name=0)
housing.index = ['Housing'] * len(housing)
housing

,Unnamed: 0,2019,2020,2021,2022,2023
Housing,DE,128.70,138.70,154.80,162.60,148.80
Housing,FR,110.86,116.62,123.98,131.82,131.26
Housing,PT,141.88,154.33,168.84,190.17,205.76


In [26]:
rental = pd.read_excel("../data/raw/week_3_project_data.xlsx", sheet_name=1)
rental.index = ['Rental'] * len(rental)
rental

,Unnamed: 0,2019,2020,2021,2022,2023
Rental,DE,100.67,101.06,101.92,102.60,104.78
Rental,FR,107.97,110.76,101.92,115.79,121.05
Rental,PT,106.10,107.60,109.00,110.80,113.10


In [28]:
income = pd.read_excel("../data/raw/week_3_project_data.xlsx", sheet_name=2)
income.index = ['Income'] * len(income)
income

,Unnamed: 0,2019,2020,2021,2022,2023
Income,DE,26105,29896,29106,28569,30308
Income,FR,26210,25382,26075,26419,27577
Income,PT,11786,12696,13113,13148,14368


In [ ]:
min_wage = pd.read_excel("../data/raw/week_3_project_data.xlsx", sheet_name=3)
min_wage.index = ["Min Wage"] * len(min_wage)

min_wage_yr = min_wage.select_dtypes(include='number')

# Multiply only numeric columns by 12
min_wage_yr = min_wage_yr * 12

min_wage_yr['Unnamed: 0'] = min_wage['Unnamed: 0']

# Reorder columns to have 'Country' as the first column
min_wage_yr = min_wage_yr[['Unnamed: 0'] + [col for col in min_wage_yr.columns if col != 'Unnamed: 0']]

min_wage_yr

In [ ]:
final_df = pd.concat([housing, rental, income, min_wage_yr], ignore_index=True)
index_labels = ['Housing'] * len(housing) + ['Rental'] * len(rental) + ['Income'] * len(income) + ["Min Wage"] * len(min_wage_yr)
final_df.index = index_labels

In [ ]:
final_df.rename(columns={"Unnamed: 0": "Country"}, inplace=True)
final_df

In [ ]:
income_df = final_df[final_df.index == 'Income']
housing_df = final_df[final_df.index == "Housing"]
rental_df = final_df[final_df.index == "Rental"]
min_df = final_df[final_df.index == "Min Wage"]

In [ ]:
income_tidy = income_df.melt(id_vars="Country", var_name="Year", value_name="Income")
housing_tidy = housing_df.melt(id_vars="Country", var_name="Year", value_name="Housing")
rental_tidy = rental_df.melt(id_vars="Country", var_name="Year", value_name="Rental")
min_tidy = min_df.melt(id_vars="Country", var_name="Year", value_name="Min Wage")


HOUSING RENTAL INCOME GRAPHS
---

In [ ]:
# Assuming income_tidy, housing_tidy, rental_tidy, and min_tidy are your original DataFrames

# 1. Add a new column to each DataFrame to indicate the type of data
income_tidy['Type'] = 'Income'
housing_tidy['Type'] = 'Housing'
rental_tidy['Type'] = 'Rental'
min_tidy['Type'] = 'Min Wage'

# 2. Concatenate the dataframes into one long dataframe
income_min_wage = pd.concat([income_tidy[['Year', 'Country', 'Income', 'Type']], 
                             min_tidy[['Year', 'Country', 'Min Wage', 'Type']].rename(columns={'Min Wage': 'Income'})], 
                            axis=0, ignore_index=True)

# 3. Replace NaN values with 0 (if any) and create the combined_data DataFrame
combined_data = pd.concat([income_min_wage, housing_tidy[['Year', 'Country', 'Housing', 'Type']],
                           rental_tidy[['Year', 'Country', 'Rental', 'Type']]], axis=0, ignore_index=True)

# 4. Create a figure and 3 subplots (axes)
fig, axes = plt.subplots(1, 3, figsize=(18, 6))  # 1 row, 3 columns

# 5. Plot the Income and Min Wage graph together
sns.lineplot(data=income_min_wage, x="Year", y="Income", hue="Country", style="Type", markers=["*", "o"], ax=axes[0])
axes[0].set_title('Income and Min Wage Over Time')
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Value')
axes[0].legend(title='Country/Type')

# 6. Plot the Housing graph
sns.lineplot(data=housing_tidy, x="Year", y="Housing", hue="Country", marker="*", ax=axes[1])
axes[1].set_title('House Price Index Over Time')
axes[1].set_xlabel('Year')
axes[1].set_ylabel('HPI')
axes[1].legend(title='Country')

# 7. Plot the Rental graph
sns.lineplot(data=rental_tidy, x="Year", y="Rental", hue="Country", marker="*", ax=axes[2])
axes[2].set_title('Rental Price Index Over Time')
axes[2].set_xlabel('Year')
axes[2].set_ylabel('RPI')
axes[2].legend(title='Country')

# 8. Adjust the x-axis to only show whole years (no 0.5 steps)
for ax in axes:
    ax.set_xticks(range(int(min(combined_data['Year'])), int(max(combined_data['Year'])) + 1))

# 9. Adjust the layout for better spacing
plt.tight_layout()  # This is the correct method

# Show the plot
plt.show()

GERMANY, BERLIN
---

In [ ]:
df_Cities = pd.read_excel("../data/raw/numbeo_stats.xlsx")
df_Cities.rename(columns = {"Unnamed: 0" : "Category"}, inplace = True)
# Fill missing 'Category' values using forward fill
df_Cities['Category'] = df_Cities['Category'].fillna(method='ffill')

df_berlin = df_Cities[df_Cities['City'] == 'Berlin']
df_berlin = df_berlin.drop(7)

# Display the updated DataFrame
df_berlin


In [ ]:
# Filter the DataFrame for Berlin
df_berlin_long = df_berlin.melt(id_vars=['Category', 'City'], var_name='Year', value_name='Cost in Euros')

# Convert 'Year' to numeric
df_berlin_long['Year'] = pd.to_numeric(df_berlin_long['Year'], errors='coerce')

# Line plot to show trends over time for each Category in Berlin
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_berlin_long, x="Year", y="Cost in Euros", hue="Category", markers=True)
plt.title('Rent Price Trends Over Time (Berlin)')
plt.xlabel('Year')
plt.ylabel('Cost in Euros')
plt.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:

# Filter the DataFrame for Berlin
df_berlin_long = df_berlin.melt(id_vars=['Category', 'City'], var_name='Year', value_name='Cost in Euros')

# Convert 'Year' to numeric
df_berlin_long['Year'] = pd.to_numeric(df_berlin_long['Year'], errors='coerce')
# Assuming df_berlin_long has already been created from df_berlin
# First, extract the minimum wage data from df_countries and melt it to long format
df_min_wage = df_countries[df_countries['Category'] == 'Min wage (after tax)'].melt(
    id_vars=['Category', 'Country'], 
    var_name='Year', 
    value_name='Cost in Euros'
)

# Filter for 'DE' (Germany)
df_min_wage_de = df_min_wage[df_min_wage['Country'] == 'DE']

# Convert 'Year' to numeric (same as df_berlin_long)
df_min_wage_de['Year'] = pd.to_numeric(df_min_wage_de['Year'], errors='coerce')

# Add 'City' as a column to match with df_berlin_long (assuming Berlin is the city)
df_min_wage_de['City'] = 'Berlin'

# Concatenate df_berlin_long with df_min_wage_de
df_combined = pd.concat([df_berlin_long, df_min_wage_de[['Category', 'City', 'Year', 'Cost in Euros']]])

# Line plot to show trends over time for each Category in Berlin
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_combined, x="Year", y="Cost in Euros", hue="Category", markers=True)
plt.title('Cost Trends Over Time in Berlin')
plt.xlabel('Year')
plt.ylabel('Cost in Euros')
plt.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


GERMANY
---

In [ ]:
df_Countries = pd.read_excel("../data/raw/numbeo_stats.xlsx", sheet_name = 1)
df_Countries.rename(columns = {"Unnamed: 0" : "Category"}, inplace = True)
# Fill missing 'Category' values using forward fill
df_Countries['Category'] = df_Countries['Category'].fillna(method='ffill')
df_Countries.columns = df_Countries.columns.str.strip()

df_Germany = df_Countries[df_Countries["Country"] == 'DE']


# Display the updated DataFrame
df_Germany


df_Germany_long = df_Germany.melt(id_vars=['Category', 'Country'], var_name='Year', value_name='Rent Price')

# Convert 'Year' to numeric (to ensure it's treated as a number)
df_Germany_long['Year'] = pd.to_numeric(df_Germany_long['Year'], errors='coerce')

# Replace the 'Country' column value from 'DE' to 'Germany' if needed
# For now, we keep it as 'DE' for simplicity.
df_Germany_long['Country'] = df_Germany_long['Country'].replace({'DE': 'Germany'})

# Line plot to show trends over time for each Category in Germany
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_Germany_long, x="Year", y="Rent Price", hue="Category", markers=True)
plt.title('Rent Price Trends Over Time (Germany)')
plt.xlabel('Year')
plt.ylabel('Rent Price')
plt.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()





# Melt Berlin's data to long format
df_berlin_long = df_berlin.melt(id_vars=['Category', 'City'], var_name='Year', value_name='Rent Price')
df_berlin_long['Year'] = pd.to_numeric(df_berlin_long['Year'], errors='coerce')

# Filter Germany's data
df_germany = df_Countries[df_Countries['Country'] == 'DE']

# Melt Germany's data to long format
df_germany_long = df_germany.melt(id_vars=['Category', 'Country'], var_name='Year', value_name='Rent Price')
df_germany_long['Year'] = pd.to_numeric(df_germany_long['Year'], errors='coerce')

# Create a side-by-side graph using matplotlib
fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

# Plot for Berlin
sns.lineplot(data=df_berlin_long, x="Year", y="Rent Price", hue="Category", markers=True, ax=axes[0])
axes[0].set_title('Rent Price Trends Over Time (Berlin)')
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Rent Price')

# Plot for Germany
sns.lineplot(data=df_germany_long, x="Year", y="Rent Price", hue="Category", markers=True, ax=axes[1])
axes[1].set_title('Rent Price Trends Over Time (Germany)')
axes[1].set_xlabel('Year')

# Adjust layout for better spacing
plt.tight_layout()

# Display t


# not working so far

# Clean column names and fill missing 'Category' values
df_Countries.rename(columns={"Country": "Region"}, inplace=True)
df_Countries['Category'] = df_Countries['Category'].fillna(method='ffill')

# Switch 'Region' back to 'Country'
df_Countries.rename(columns={"Region": "Country"}, inplace=True)

# Filter Berlin's data
df_berlin = df_Countries[df_Countries['City'] == 'Berlin']

# Filter out the "Buy apartment" category from Berlin's data
df_berlin = df_berlin[df_berlin['Category'] != 'Buy apartment ( per m2 in city center)']

# Melt Berlin's data to long format
df_berlin_long = df_berlin.melt(id_vars=['Category', 'City'], var_name='Year', value_name='Rent Price')
df_berlin_long['Year'] = pd.to_numeric(df_berlin_long['Year'], errors='coerce')

# Filter Germany's data
df_germany = df_Countries[df_Countries['Country'] == 'DE']

# Filter out the "Buy apartment" category from Germany's data
df_germany = df_germany[df_germany['Category'] != 'Buy apartment ( per m2 in city center)']

# Melt Germany's data to long format
df_germany_long = df_germany.melt(id_vars=['Category', 'Country'], var_name='Year', value_name='Rent Price')
df_germany_long['Year'] = pd.to_numeric(df_germany_long['Year'], errors='coerce')

# Create a side-by-side graph using matplotlib
fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

# Plot for Berlin
sns.lineplot(data=df_berlin_long, x="Year", y="Rent Price", hue="Category", markers=True, ax=axes[0])
axes[0].set_title('Rent Price Trends Over Time (Berlin)')
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Rent Price')

# Plot for Germany
sns.lineplot(data=df_germany_long, x="Year", y="Rent Price", hue="Category", markers=True, ax=axes[1])
axes[1].set_title('Rent Price Trends Over Time (Germany)')
axes[1].set_xlabel('Year')

# Adjust layout for better spacing
plt.tight_layout()

# Display the plots
plt.show()




# Melt the DataFrame to long format for easier plotting
df_long = df_numbeo.melt(id_vars=['Category', 'City'], var_name='Year', value_name='Rent Price')

# Convert 'Year' to numeric
df_long['Year'] = pd.to_numeric(df_long['Year'], errors='coerce')

# Line plot to show trends over time for each Category and City
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_long, x="Year", y="Rent Price", hue="Category", style="City", markers=True)
plt.title('Rent Price Trends Over Time')
plt.xlabel('Year')
plt.ylabel('Rent Price')
plt.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()
